In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import re
import pickle

import requests
import bs4
import codecs
from codecs import open

from sentiment_classifier import SentimentClassifier
import time
from flask import Flask, render_template, request
from nocache import nocache

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [4]:
app = Flask(__name__)

print ("Подготовка классификатора...")
start_time = time.time()
classifier = SentimentClassifier()
print ("Классификатор готов")
print (time.time() - start_time, "секунд")

@app.route("/", methods=["POST", "GET"])
@nocache
def index_page(text="", prediction_message=""):
    if request.method == "POST":
        text = request.form["text"]
        print (text)
        prediction_message = classifier.get_prediction_message(text)
        print (prediction_message)
    return render_template('index.html', text=text, prediction_message=prediction_message)

if __name__ == "__main__":
    app.run()

Подготовка классификатора...
Классификатор готов
1.0506982803344727 секунд


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Apr/2018 16:24:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:30] "GET /static/css/bootstrap.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:30] "GET /static/css/fontawesome.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:30] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:30] "GET /static/js/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:30] "GET /static/js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:30] "GET /static/img/04.jpg HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:30] "GET /static/img/1_pos.jpg HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:30] "GET /static/img/2_neg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:31] "GET /static/img/3_prob_neg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:31] "GET /static/img/regression.jpg HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2018 16:24:31] "GET /static/img/C

In [3]:
# код сбора данных для конкретной странички в отдельную функцию
def get_data(form, url):
    text = requests.get(url).text
    parser = bs4.BeautifulSoup(text, 'lxml')
    class_str = r'\"review\"' # r в начале означает, что спецсимволы типа \,",\n не будут конвертироваться в новые строки
    reviews = parser.findAll('div', attrs={'class':class_str})
    
    if reviews == None:
        print ('Ошибка: по данному адресу ' + url + 'нет отзывов!')
        return None
    
    texts = []
    labels = []
    # парсим отзывы на тексты и метки
    for i in range(len(reviews)):
        st = reviews[i].find('img')
        if st != None:
            idx = str(st).find('smile')
            if idx > 0:
                class_str = r'\'review-title\''
                title = reviews[i].find('div', attrs={'class':class_str})
                class_str = r'\'review-comment\''
                comment = reviews[i].find('div', attrs={'class':class_str})
                class_str = r'\'review-plus\''
                plus = reviews[i].find('div', attrs={'class':class_str})
                class_str = r'\'review-minus\''
                minus = reviews[i].find('div', attrs={'class':class_str})
                class_str = r'\'review-period\''
                period = reviews[i].find('div', attrs={'class':class_str})
                
                label = int(str(st)[idx+6:idx+7])
                
                try:
                    if (label == 1 or label == 2) and (form != 'pos'): # если отзыв негативный, то плюсы игнорируем
                        text = []
                        if title != None: text = title.text.lower()
                        if comment != None: text = text + ' ' + comment.text.lower()
                        if minus != None: text = text + ' ' + minus.text.lower()
                        #if plus != None: text = text + ' ' + plus.text.lower()
                        if period != None: text = text + ' ' + period.text.lower()
                        texts.append(text)
                        labels.append(0)
                    elif (label == 3 or label == 4) and (form != 'neg'): # если позитивный, то минусы игнорируем
                        text = []
                        if title != None: text = title.text.lower()
                        if comment != None: text = text + ' ' + comment.text.lower()
                        #if minus != None: text = text + ' ' + minus.text.lower()
                        if plus != None: text = text + ' ' + plus.text.lower()
                        if period != None: text = text + ' ' + period.text.lower()
                        texts.append(text)
                        labels.append(1)
                except:
                    print (i)
                    continue

    return texts, labels

In [9]:
# перемешиваем 2 массива синхронно
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [10]:
# собираем вручную id страниц с отзывами и получаем данные, состоящие из 3000 отзывов для обучения модели
#%%time
texts = []
labels = []

# все отзывы
ids = ['451560', '477190', '756658', '970070', '311531', '970462', '1096093', 
      '100538', '85658', '60787', '260239', '82577', '77604', '47888', '77601', '103345', '103346', '157759', '88871', '96704',
       '58222', '58224', '54176', '244246', '820213', '543531', '625504', '310279', '846154', '910896']

for i in range(len(ids)):
    t, l = get_data('all', 'http://www.e-katalog.ru/mtools/dot_output/mui_review.php?idg_=' + ids[i] + '&p_start_=1&p_end_=1000')
    texts.extend(t)
    labels.extend(l)

print ('Всего отзывов:', len(labels))
print ('Доля положительных: ', round(np.mean(labels), 2))

# перемешиваем
texts_train, labels = shuffle_in_unison(np.asarray(texts), np.asarray(labels))

cv = 5

Всего отзывов: 2879
Доля положительных:  0.86


In [11]:
# Векторайзер и модель
tf_vectorizer = CountVectorizer(ngram_range=(3, 5), analyzer=u'char')
mx = tf_vectorizer.fit_transform(texts_train)
model = LogisticRegression(class_weight='balanced')
model.fit(mx, labels)

# сериализация векторайзера и модели в бинарные файлы
with open('./LogisticUnigramUnprocessedTextSentiment.pkl', 'wb') as handle:
    pickle.dump(model, handle)
with open('./UnigramUnprocessedVectorizer.pkl', 'wb') as handle:
    pickle.dump(tf_vectorizer, handle)

In [64]:
class SentimentClassifier_(object):
    def __init__(self):
        with open('./LogisticUnigramUnprocessedTextSentiment.pkl', 'rb') as f:
            self.model = pickle.load(f)
        with open('./UnigramUnprocessedVectorizer.pkl', 'rb') as f:
            self.vectorizer = pickle.load(f)
        self.classes_dict = {0: u"негативный", 1: u"позитивный", -1: u"ошибка прогноза"}

    @staticmethod
    def get_probability_words(probability):
        if probability < 0.55:
            return u"нейтральный или неточный"
        if probability < 0.71:
            return u"скорее всего"
        if probability > 0.95:
            return u"точно"
        else:
            return ""

    def predict_text(self, text):
        try:
            vectorized = self.vectorizer.transform([text])
            return self.model.predict(vectorized)[0],\
                self.model.predict_proba(vectorized)[0].max()
        except:
            print (u"ошибка прогноза")
            return -1, 0.8

    def predict_list(self, list_of_texts):
        try:
            vectorized = self.vectorizer.transform(list_of_texts)
            return self.model.predict(vectorized),\
                self.model.predict_proba(vectorized)
        except:
            print (u"ошибка прогноза")
            return None

    def get_prediction_message(self, text):
        prediction = self.predict_text(text)
        class_prediction = prediction[0]

        prediction_probability = prediction[1]
        if (prediction_probability > 0.65) and (prediction_probability < 0.71):
            class_prediction = 0

        return self.get_probability_words(prediction_probability) + " " + self.classes_dict[class_prediction]

In [12]:
#test_review = 'очень плохой телефон. Не рекоммендую покупать его'
#test_review = 'очень хороший телефон. Обязателен к покупке!'
#test_review = 'Считаю его одним из лучших в своем ценовом диапазоне. Очень шустрый телефон, приятная оболочка ( лучше гуголовской), хорошо сидит в руке. Отдельно хотел похвалить аккумулятор - телефон спокойно хватает на два дня при среднем времени использования. Единственным недостатком считаю камеру, так как она плохо фокусируется и часто все мажет'
#test_review = 'Ужасная камера, отвратительная просто. Звук громкости, тихий очень. Подсветки на клавишах нет. Индикатора пропущенных вызовов нет. Не стоит таких денег, как на витрине. Хуже китайцев. А, но да за марку самсунг денег же содрать нужно. Фуфло полное. Кто думает его покупать. Сначала хорошенько все посмотрите и подумайте. Удачи!!!'
#test_review = 'Взял по вроде не плохим параметрам: батарея, камера, 4G, 1ГБ оперативки. До этого имел опыт эксплуатации девайсов марки престижио и в принципе был хорошего мнения о них. Но этот аппарат совсем меня как то растроил всем, чем только можно. Да аккум держит подольше моего прежнего алкателя, но не сильно, хотя 5000мАч против 2000мАч должен как минимум раза в 2 держать заряда дольше, все приложения да и любые действия на аппарате происходят с такой задержкой...., что кажется тут максимум андроид 2 стоит не больше. Ну и пожалуй самый главный косяк, это слабый микрофон-(собеседник плохо слышит)'
#test_review = 'Не могу нарадоваться на этот телефон!'
#test_review = 'Опять снег пошел. Вот тебе и весна :('
test_review = 'отсутствие памяти, камера влюбилась в этот телефон за его внешний вид, целенаправленно искала нужную расцветку, не обращая внимания на плохие отзывы. зря! внешне он меня до сих пор радует, но пользоваться им - сплошное мучение. карта памяти есть, но фото на нее не сохраняются, если памяти на самом телефоне мало (где связь?). приложений осталось 2-3 штуки, но и им недолго осталось, скоро и их придется удалить.'
classifier = SentimentClassifier()
prediction_message = classifier.get_prediction_message(test_review)
print (prediction_message)

скорее всего негативный
